In [ ]:
# !pip install geopandas
# !pip install pygeos
# !pip install gpdvega

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import collections

from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import silhouette_samples
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from scipy.cluster.hierarchy import ward, dendrogram, complete, average, single

# import geopandas as gpd
# import pygeos
# import gpdvega 

import altair as alt

In [ ]:
import os
if 'COLAB_GPU' in os.environ:
    from google.colab import  drive
    drive.mount('/drive')
    data_path = '/drive/Shared drives/Capstone/notebooks/data'
else:
    data_path = 'data'

In [ ]:
#path = f'{data_path}/processed/counties_merged_FIPS_str.csv'

path = f'{data_path}/processed/counties_merged.csv'
df = pd.read_csv(path,   dtype={"fips": str})
df['FIPS'] = df['FIPS'].astype(int).astype(str).str.zfill(5)

df['population'].dropna(inplace=True)
df['HPSA Score'] = df['HPSA Score'].fillna(0)

df.head()

In [ ]:
# df['population']

# cols = ['white', 'black', 'native_american', 'asian', 'hawaiian',
#        'some_other_race_alone', 'two_more_races', 'hispanic_or_latino',
#        'median_family_income', 'income_20_percentile', 'income_80_percentile',
#        'median_family_income_white', 'median_family_income_black',
#        'median_family_income_indigenous', 'median_family_income_asian',
#        'median_family_income_hispanic', 'preschool_enrollment_white',
#        'preschool_enrollment_black', 'preschool_enrollment_hispanic',
#        'preschool_enrollment_indigenous', 'preschool_enrollment_asian',
#        'all_in_poverty', 'year', 'public_students_pre_12',
#        'white_employed_16_64', 'black_employed_16_64',
#        'american_indian_employed_16_64', 'asian_employed_16_64',
#        'some_other_race_alone_employed_16_64',
#        'two_or_more_race_employed_16_64', 'hispanic_or_latino_employed_16_64',
#        'employed_25_54_population', 'employed_16_64_population',
#        'preschool_enroll', 'white_under_5', 'black_under_5',
#        'indigenous_under_5', 'asian_under_5', 'hispanic_under_5',
#        'two_or_more_race_under_5', 'some_other_race_under_5',
#        'avg_edu_prof_diff', 'low_birth_rate',
#        'Not Hispanic or Latino_low_birth_rate',
#        'Hispanic or Latino_low_birth_rate',
#        'Unknown or Not Stated_low_birth_rate',
#        'Black or African American_low_birth_rate', 'White_low_birth_rate',
#        'Asian_low_birth_rate', 'More than one race_low_birth_rate',
#        'American Indian or Alaska Native_low_birth_rate',
#        'Native Hawaiian or Other Pacific Islander_low_birth_rate',
#        'HPSA Score', 'HOM_STUDENTS', 'proportion_homeless', 'proportion_voter',
#        'transit_trips_index', 'transit_low_cost_index', 'final_2019_crimes',
#        'crime_rate', 'Total_Juvenile_Arrests', 'juvenile_crime_rate',
#        'hispanic_or_latino_exposure', 'white_exposure', 'black_exposure',
#        'native_american_exposure', 'asian_exposure', 'hawaiian_exposure',
#        'some_other_race_alone_exposure', 'two_more_races_exposure',
#        'proportion_high_poverty_neighborhood']
#cols = ['some_other_race_alone_exposure']

# create weighted average for each of the metrics
# weighted = []
# weighted2 = []
# not_weighted = []
# for col in cols:
#   weighted.append((df['population']*df[col]/(df['population'].sum())).sum())
#   not_weighted.append(df[col].mean())

# df2 = pd.DataFrame(
#     {'metric': cols,
#      'weighted': weighted,
#      'not_weighted': not_weighted
#     })

# df2.to_csv(f'{data_path}/interim/weighted_and_unweighted.csv', index=False)

In [ ]:
cols = ['median_family_income', 'income_20_percentile', 'income_80_percentile',
       'median_family_income_white', 'median_family_income_black',
       'median_family_income_hispanic', 
       'white_employed_16_64', 'black_employed_16_64',
       'hispanic_or_latino_employed_16_64',
       'employed_25_54_population',
       'preschool_enroll', 
       'avg_edu_prof_diff', #'low_birth_rate', too many missing values
       'HPSA Score', 'HOM_STUDENTS', 'proportion_homeless', 'proportion_voter',
       'transit_trips_index', 'transit_low_cost_index',
       'crime_rate', 'juvenile_crime_rate','debt_all', 'AQI',
       'proportion_high_poverty_neighborhood']

#cols = ['proportion_high_poverty_neighborhood', 'white_exposure', 'black_exposure', 'transit_trips_index', 'transit_low_cost_index', 'AQI','crime_rate', 'juvenile_crime_rate']
df[cols]=df[cols].fillna(df.mean().iloc[0])

In [ ]:
county_list_mapper = list(df['FIPS'])

county_dict_mapper = (df[['FIPS']].to_dict('index'))


In [ ]:

#Identify and drop non-numeric fields
#non_numeric = df.select_dtypes(exclude='number').columns.to_list()
#us_df = df.drop(columns=non_numeric)


#Remove fields that are numeric, but not meaningful features of the county
us_df=df[cols]
#us_df = us_df.drop(columns = {'state','county','year'})
us_df

In [ ]:
print(us_df.shape)
print(len(county_dict_mapper))

{k:v for (k,v) in county_dict_mapper.items() if k < 5}

In [ ]:
#Store the column names for later use
us_df_columns = us_df.columns
us_df_columns

us_df.head()

### Feature Scaling
We'll use min-max normalization since many of our features are on different scales, and also numeric. May be worth changing this later

In [ ]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(us_df)

# verify that there are no missing values and inspect data
pd.DataFrame(scaled_data).isna().sum().sum()



In [ ]:
scaled_data.shape

### K-Means Clustering

We will run K-Means with multiple n_clusters values so we can use the elbow method. To determine the optimal number of clusters, we have to select the value of k at the “elbow”, or the point after which the inertia start decreasing in a linear fashion. 

Inertia is the sum of squared distances of samples to their closest cluster center (SSE)

We will try two methods of initializing K-Means, the default, which is 'k-means++', which selectes inital cluster centers in a "smart way", and 'random' which choses initial centroids at random.

We will also look at two metrics to check the quality of our clusters:

#### Davies_Bouldin Index
This index signifies the average ‘similarity’ between clusters, where the similarity is a measure that compares the distance between clusters with the size of the clusters themselves. Zero is the lowest possible score. Values closer to zero indicate a better partition.

#### Calinski-Harabasz Index
The index is the ratio of the sum of between-clusters dispersion and of within-cluster dispersion for all clusters (where dispersion is defined as the sum of distances squared). The score is higher when clusters are dense and well separated, which relates to a standard concept of a cluster.

In [ ]:
k_range = [*range(3, 10)]

#Calculate inertia scores using the default initialization 'k-means++'
inertia_scores = []
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42,init ='k-means++').fit(scaled_data)
    inertia_scores.append(kmeans.inertia_)
    kmeans.fit(scaled_data)
    ch_score = metrics.calinski_harabasz_score(scaled_data,kmeans.labels_)
    db_score = metrics.davies_bouldin_score(scaled_data,kmeans.labels_)
    print('k: {}, ch_score: {}, db_score {} '.format(k,ch_score,db_score))


In [ ]:
plt.plot(k_range, inertia_scores, '-o')
plt.ylabel('SSE')
plt.xlabel('Number of clusters (k)')
plt.title('The Elbow Method using "k-means++" initialization ')

It looks like the elbow occurs at around 4 clusters

Let's try using silhouette analysis for 3-8 clusters.

From https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py:

Silhouette analysis can be used to study the separation distance between the resulting clusters. The silhouette plot displays a measure of how close each point in one cluster is to points in the neighboring clusters and thus provides a way to assess parameters like number of clusters visually. This measure has a range of [-1, 1].

Silhouette coefficients (as these values are referred to as) near +1 indicate that the sample is far away from the neighboring clusters. A value of 0 indicates that the sample is on or very close to the decision boundary between two neighboring clusters and negative values indicate that those samples might have been assigned to the wrong cluster.

In [ ]:
# Code from https://towardsdatascience.com/k-means-clustering-algorithm-applications-evaluation-methods-and-drawbacks-aa03e644b48a
# Per the article, a good number of clusters will have a well above 0.5 silhouette 
# average score as well as all of the clusters have higher than the average score

for i, k in enumerate(range(3,8)):
    #fig, (ax1, ax2) = plt.subplots(1, 2)
    fig, ax1 = plt.subplots(1, 1)
    fig.set_size_inches(18, 7)
    
    # Run the Kmeans algorithm
    km = KMeans(n_clusters=k)
    labels = km.fit_predict(scaled_data)
    centroids = km.cluster_centers_

    # Get silhouette samples
    silhouette_vals = silhouette_samples(scaled_data, labels)

    # Silhouette plot
    y_ticks = []
    y_lower, y_upper = 0, 0
    for i, cluster in enumerate(np.unique(labels)):
        cluster_silhouette_vals = silhouette_vals[labels == cluster]
        cluster_silhouette_vals.sort()
        y_upper += len(cluster_silhouette_vals)
        ax1.barh(range(y_lower, y_upper), cluster_silhouette_vals, edgecolor='none', height=1)
        ax1.text(-0.03, (y_lower + y_upper) / 2, str(i + 1))
        y_lower += len(cluster_silhouette_vals)

    # Get the average silhouette score and plot it
    avg_score = np.mean(silhouette_vals)
    ax1.axvline(avg_score, linestyle='--', linewidth=2, color='green')
    ax1.set_yticks([])
    ax1.set_xlim([-0.1, 1])
    ax1.set_xlabel('Silhouette coefficient values')
    ax1.set_ylabel('Cluster labels')
    ax1.set_title('Silhouette plot for the various clusters', y=1.02);
    print('Average silhouette score for {} clusters is {}'.format(k,avg_score))
    
   # plt.tight_layout()
    plt.suptitle(f'Silhouette analysis using k = {k}',
                 fontsize=16, fontweight='semibold', y=1.05);


The highest score is for 3 clusters, depending on the run .45.
Maybe there are too many dimensions? Can try running PCA on the data, then K-Means, later. For now, lst's try another type of clustering.

Agglomerative Clustering
Agglomerative clustring performs a bottom up approach where each observation starts in its own cluster and clusters are successively merged together.

There are 4 types of linkages we will consider:

Single: minimum distance between clusters
Complete: maximum distance between clusters
Average: average distance between clusters
Ward: difference between:
The total within-cluster sum of squares for the two clusters seperately
and
The within-cluster sum of squares resulting from merging the two clusters
We are using Ward's Method to start because it tends to create equal sized clusters and is effective for noisy data.

The output of the ward function is as follows:

Column 1 and 2 are child nodes
Column 3 is distance
Column 4 is the number of leaf nodes merged

In [ ]:
plt.figure(figsize=(10,6))
cls1 = ward(scaled_data)
dendrogram(cls1)#,orientation='left')
plt.title('Dendrogram using Ward Linkage')
plt.show()

print(cls1)

Based on the dendrogram we are going to assign the data to 4 clusters, so now we will run the clusters to get the assignment value

In [ ]:
cls_w = AgglomerativeClustering(n_clusters=4, linkage='ward')
cls_assignment_w = cls_w.fit_predict(scaled_data)
cls_assignment_w

In [ ]:
county_cluster = list(zip(county_list_mapper, list(cls_assignment_w)))

cluster_df = pd.DataFrame(county_cluster, columns=['FIPS', 'agglom_ward_cluster'])
(cluster_df.head())


In [ ]:
#Zip together the mapper and the cluster assignment

#Show the clusters on a map
us_all = pd.merge(df,cluster_df,how='left',on='FIPS')

# # remove Hawaii and Alaska
# us_all = us_all[us_all['State_Name'] != 'Hawaii']
# us_all = us_all[us_all['State_Name'] != 'Alaska']

# len(us_all)

us_all.columns

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
    
us_all["agglom_ward_cluster"] = us_all["agglom_ward_cluster"].astype(str)
import plotly.express as px

fig = px.choropleth(us_all, geojson=counties, locations='FIPS', color='agglom_ward_cluster',
                           scope="usa",
                           labels={'agglom_ward_cluster':'agglom_ward_cluster'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Out of curiousity, let's see if we get similar clusterings if we use different agglomerative clustering linkage methods

In [ ]:
#Complete linkage method
plt.figure(figsize=(10,6))
cls2 = complete(scaled_data)
dendrogram(cls2)#,orientation='left')
plt.title('Dendrogram using Complete Linkage')
plt.show()

#print(cls2)

With complete linkage it seems that 12? clusters are more appropriate

In [ ]:

cls_c = AgglomerativeClustering(n_clusters=8, linkage='complete')
cls_assignment_c = cls_c.fit_predict(scaled_data)

county_cluster = list(zip(county_list_mapper, list(cls_assignment_c)))

new_df = pd.DataFrame(county_cluster, columns=['FIPS', 'agglom_complete_cluster'])

us_all = us_all.merge(new_df, how = 'left', on = 'FIPS')



In [ ]:
us_all['agglom_complete_cluster']

In [ ]:
import plotly.express as px

us_all["agglom_complete_cluster"] = us_all["agglom_complete_cluster"].astype(str)


fig = px.choropleth(us_all, geojson=counties, locations='FIPS', color='agglom_complete_cluster',
                           scope="usa",
                            #hover_data=["agglom_complete_cluster",'population', 'County_Name'],
                           #labels={'agglom_complete_cluster':'agglom_complete_cluster','population':'Population'},
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
#Average Linkage Method
plt.figure(figsize=(10,6))
cls3 = average(scaled_data)
dendrogram(cls3)#,orientation='left')
plt.title('Dendrogram using Average Linkage')
plt.show()

#print(cls3)

### PCA
We are working with a lot of features in the US. Maybe this is impacting the ability to create useful clusters. Let's try PCA to help identify the most relevant features and to reduce the dimensionality so we can try clustering again.

In [ ]:
pca = PCA(2)
pca_data = pca.fit_transform(scaled_data)

Let's plot and check the variance of the components using a variance ratio plot

In [ ]:
plt.figure(figsize=(10,10))
var = np.round(pca.explained_variance_ratio_*100, decimals = 1)
lbls = [str(x) for x in range(1,len(var)+1)]
plt.bar(x=range(1,len(var)+1), height = var, tick_label = lbls)
plt.xlabel('Principal Component')
plt.ylabel('Variance %')
plt.title('PCA Explained Variance')
plt.show()



In [ ]:
#Plot the magnitude of each feature value for the 2 principal components
fig, ax = plt.subplots(figsize=(20, 18))
plt.imshow(pca.components_[0:2].T, interpolation = 'none', cmap = 'plasma')
feature_names=list(us_df_columns)
plt.yticks(np.arange(-0., len(feature_names), 1) , feature_names, fontsize=10)
plt.xticks(np.arange(0., 2, 1), ['First PC', 'Second PC'], rotation = 90, fontsize = 16)
plt.colorbar()

In [ ]:
pca_df = pd.DataFrame(pca.components_.T, index=us_df_columns)
pca_df['PC-1'] = abs(pca_df[0])
pca_df['PC-2'] = abs(pca_df[1])

pca_df['PC-1'].nlargest(n=10)

In [ ]:
pca_df['PC-2'].nlargest(n=10)

Biplot
The cosine of the angle between any two variable markers (vectors) is the coefficient of correlation between those variables.
Note - this only holds if first two PC capture 80% of variance which is NOT the case here.

The cosine of the angle between a vector and the axis for a given PC is the coefficient of correlation between those two variables.

In [ ]:
#From SIADS 543
feature_subset_count = 10

### Feel free to use this routine to plot your own biplots!
def biplot(score, coeff, maxdim, pcax, pcay, labels=None):
    zoom = 0.5
    pca1=pcax-1
    pca2=pcay-1
    xs = score[:,pca1]
    ys = score[:,pca2]
    n = min(coeff.shape[0], maxdim)
    width = 2.0 * zoom
    scalex = width/(xs.max()- xs.min())
    scaley = width/(ys.max()- ys.min())
    text_scale_factor = 1.3
        
    fig = plt.gcf()
    fig.set_size_inches(9, 9)
    
    plt.scatter(xs*scalex, ys*scaley, s=9)
    for i in range(n):
        plt.arrow(0, 0, coeff[i,pca1], coeff[i,pca2],
                  color='b',alpha=0.9, head_width = 0.03 * zoom) 
        if labels is None:
            plt.text(coeff[i,pca1]* text_scale_factor, 
                     coeff[i,pca2] * text_scale_factor, 
                     "Var"+str(i+1), color='g', ha='center', va='center')
        else:
            plt.text(coeff[i,pca1]* text_scale_factor, 
                     coeff[i,pca2] * text_scale_factor, 
                     labels[i], color='black', ha='center', va='center')
    
    plt.xlim(-zoom,zoom)
    plt.ylim(-zoom,zoom)
    plt.xlabel("PC{}".format(pcax))
    plt.ylabel("PC{}".format(pcay))
    plt.grid()

plt.figure()

feature_subset = slice(0, feature_subset_count, 1)

biplot(pca_data, np.transpose(pca.components_[0:2, feature_subset]), 
       feature_subset_count, 1, 2, labels=feature_names[feature_subset])

print("explained_variance_ratio:", pca.explained_variance_ratio_)
print("sum of explained variance ratios:", np.sum(pca.explained_variance_ratio_))
print("singular values:", pca.singular_values_)  


### K-Means after PCA

Now we will train our model based on the new features generated by PCA

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=52).fit(scaled_data)

In [ ]:
#NOTE - for some reason, you need to run this again seperately in order for the cluster centers
# to print in the correct spot
centers = np.array(kmeans.cluster_centers_)
#print(centers)

pca_label = kmeans.fit_predict(pca_data)
plt.figure(figsize=(10,10))
uniq = np.unique(pca_label)
for i in uniq:
   plt.scatter(pca_data[pca_label == i , 0] , pca_data[pca_label == i , 1] , label = i)
plt.scatter(centers[:,0], centers[:,1], marker="x", color='k')
#This is done to find the centroid for each clusters.
plt.legend()
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

In [ ]:

county_cluster = list(zip(county_list_mapper, list(pca_label)))

new_df = pd.DataFrame(county_cluster, columns=['name', 'kmeans_pca_cluster'])
new_df

us_all = us_all.merge(new_df, how='left',left_on='FIPS',right_on='name')
us_all.columns

In [ ]:
us_all.head()

In [ ]:
#Show K-Means after PCA clusters map


fig = px.choropleth(us_all, geojson=counties, locations='FIPS', color='kmeans_pca_cluster',
                           scope="usa",
                            #hover_data=["agglom_complete_cluster",'population', 'County_Name'],
                           #labels={'agglom_complete_cluster':'agglom_complete_cluster','population':'Population'},
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


In [ ]:
us_all.head()

In [ ]:
import seaborn as sns

fig, ax = plt.subplots()
sns.violinplot(x='kmeans_pca_cluster', y='transit_low_cost_index',
                    data=us_all, ax=ax)

In [ ]:
fig, ax = plt.subplots()
sns.violinplot(x='kmeans_pca_cluster', y='transit_trips_index',
                    data=us_all, ax=ax)


In [ ]:
sns.histplot(data=us_all, x="AQI")